In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind

base = pd.read_csv('main_task.csv')

col = ['Restaurant_id', 'City', 'Cuisine_Style', 'Ranking',
       'Rating', 'Price_Range', 'Number_of_Reviews', 'Reviews', 'URL_TA', 'ID_TA']
base.columns = col
# переименовали столбы для удобства

base

,Restaurant_id,City,Cuisine_Style,Ranking,Rating,Price_Range,Number_of_Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963
...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,$$ - $$$,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414
39996,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,$$ - $$$,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036
39997,id_1649,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,NaN,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615
39998,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,$$ - $$$,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838


In [103]:
# проверяем столбец с Restaurant_id (идентификационный номер ресторана / сети ресторанов)

base.Restaurant_id.value_counts()
# пропусков нет, все ок
base.Restaurant_id.describe()
# по условию - в одному id  могут быть прикреплены несколько ресторанов одной сети,
# поэтому уникальных значений меньше чем записей

base.Restaurant_id = [i[3:] for i in base.Restaurant_id]
base.Restaurant_id = [int(i) for i in base.Restaurant_id]

len(base.Restaurant_id)

# превратили столбец в численное выражение

40000

In [104]:
# проверяем столбец с City (город, в котором находится ресторан)

base.City.value_counts()
# пропусков нет, города не повторяются, проблем с ракладкой или регистром нет, все ок


A = ['London', 'Paris', 'Madrid', 'Barcelona', 'Berlin', 'Milan', 'Rome', 'Prague', 'Lisbon', 'Vienna', 'Amsterdam',
     'Brussels', 'Hamburg', 'Munich', 'Lyon', 'Stockholm', 'Budapest', 'Warsaw', 'Dublin', 'Copenhagen', 'Athens',
     'Edinburgh', 'Zurich', 'Oporto', 'Geneva', 'Krakow', 'Oslo', 'Helsinki', 'Bratislava', 'Luxembourg', 'Ljubljana']
B = range(0, len(A))

base['City'] = base['City'].replace(A, B)
len(base.City)

# аномалий нет + мы заменили города на числовые индефикаторы

40000

In [105]:
# проверяем столбец с Cuisine_Style (кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане)

base['Cuisine_Style'] = base['Cuisine_Style'].fillna('""No_info"')
# заменяем пропуски

new = pd.DataFrame(base.Cuisine_Style.dropna())
a = list(new.Cuisine_Style)
b = list()


def l(x):
    i = 0
    for g in x:
        f = x[i].split(',')
        v = 0
        for g in f:
            h = f[v][2:-1].replace("'", '')
            v = +1
            b.append(h)
        i += 1


l(a)
b = set(b)
b = list(b)
# нашли список уникальных значений кухонь


def find_item(cell):
    if item in cell:
        return 1
    return 0


for item in b:
    base[item] = base['Cuisine_Style'].apply(find_item)

base['Cuisine_Style'] = base['Cuisine_Style'].apply(lambda x: len(x))

len(base['Cuisine_Style'])
# пропуски заменены, строки преобразованы в столбцы

40000

In [106]:
# проверяем столбец с Price_Range (диапазон цен в ресторане)


base['Price_Range'] = base['Price_Range'].replace('$', 1)
base['Price_Range'] = base['Price_Range'].replace('$$ - $$$', 2)
base['Price_Range'] = base['Price_Range'].replace('$$$$', 3)
#base['Price_Range']=base['Price_Range'].replace ('no' ,0)

a = base['Price_Range'].mean()
a = int(a)
base['Price_Range'] = base['Price_Range'].fillna(a)


print(base.Price_Range.value_counts())

# заменили буквенные индефикаторы на численные

1.0    20165
2.0    18412
3.0     1423
Name: Price_Range, dtype: int64


In [107]:
# проверяем столбец с Reviews (данные о двух отзывах, которые отображаются на сайте ресторана)

base['Reviews'] = base.Reviews.replace("[[], []]", 'No_info')
# заменяем пропуски

base.Last_rew = [i[-27:-17] for i in base.Reviews]
# добавляем на основе столбца новую информацию - дату последнего отзыва

base

/Users/yasckevich/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


,Restaurant_id,City,Cuisine_Style,Ranking,Rating,Price_Range,Number_of_Reviews,Reviews,URL_TA,ID_TA,...,Portuguese,Indonesian,Barbecue,Taiwanese,Filipino,Korean,Halal,Mexican,Israeli,Street Food
0,5569,1,39,5570.0,3.5,2.0,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,...,0,0,0,0,0,0,0,0,0,0
1,1535,15,10,1537.0,4.0,1.0,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,...,0,0,0,0,0,0,0,0,0,0
2,352,0,102,353.0,4.5,3.0,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,...,0,0,0,0,0,0,0,0,0,0
3,3456,4,10,3458.0,5.0,1.0,3.0,No_info,/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,...,0,0,0,0,0,0,0,0,0,0
4,615,13,53,621.0,4.0,2.0,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,499,5,74,500.0,4.5,2.0,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414,...,0,0,0,0,0,0,0,0,0,0
39996,6340,1,64,6341.0,3.5,2.0,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036,...,0,0,0,0,0,0,0,0,0,0
39997,1649,15,21,1652.0,4.5,1.0,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615,...,0,0,0,0,0,0,0,0,0,0
39998,640,17,85,641.0,4.0,2.0,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838,...,0,0,0,0,0,0,0,0,0,0


In [108]:
# проверяем столбец с Number_of_Reviews (количество отзывов о ресторане)
base['Number_of_Reviews'].describe()
# отрицательных чисел нет

base['Number_of_Reviews'].value_counts()
# но есть пропуски, нужно проверить, что где есть пропуски - там действительно нет отзывов

base['Number_of_Reviews'] = base['Number_of_Reviews'].fillna('No_info')
# меняем пропуски на Not_info

a = base.query('Reviews == "No_info" & Number_of_Reviews == "No_info"')
b = list(a.index)
base['Number_of_Reviews'][b] = 0
# где нет значения кол-ва отзывов и превью отзывов - ставим 0

len(base['Number_of_Reviews'])
# заменили предполагаемые пропуски, в некоторые смогли подставить значения. С 2,5+ тысяч снизили пропуски до 900+


base = base.drop(np.where(base['Number_of_Reviews'] == 'No_info')[0])
# удаляем пропуски, которые не смогли заполнить
base['Number_of_Reviews'].value_counts()

/Users/yasckevich/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


2.0       1916
3.0       1636
0.0       1597
4.0       1370
5.0       1181
          ... 
1150.0       1
1146.0       1
1145.0       1
1144.0       1
738.0        1
Name: Number_of_Reviews, Length: 1460, dtype: int64

In [109]:
# проверяем м столбец с URL_TA (URL страницы ресторана на TripAdvosor)

base['URL_TA'].value_counts()

# очень странно, что сслылки повторяются
# посмотрим что с ними на примере одного ресторана
base.query(
    'URL_TA == "/Restaurant_Review-g187514-d697406-Reviews-Al_Mounia-Madrid.html"')
# совпадают 8 столбцов из 10 основных

# удалим дубликаты по стобцам сайт, ID на сайте, город, кол-во отзывов, кухни и средний чек
base = base.drop_duplicates(
    subset=['City', 'Reviews', 'URL_TA', 'ID_TA', 'Cuisine_Style', 'Price_Range'])

base['URL_TA'].value_counts()

# дубликаты удалены

/Restaurant_Review-g187514-d12596349-Reviews-Pasteleria_Salamat-Madrid.html                             1
/Restaurant_Review-g189158-d2277303-Reviews-Bica_e_Agua-Lisbon_Lisbon_District_Central_Portugal.html    1
/Restaurant_Review-g186338-d730271-Reviews-My_Old_Dutch_Pancake_House-London_England.html               1
/Restaurant_Review-g186338-d7222373-Reviews-The_Camberwell_Arms-London_England.html                     1
/Restaurant_Review-g187323-d951221-Reviews-Zeitlos-Berlin.html                                          1
                                                                                                       ..
/Restaurant_Review-g274707-d4768736-Reviews-Ambrozie-Prague_Bohemia.html                                1
/Restaurant_Review-g188113-d10124292-Reviews-Schonau_Bar_Restaurant-Zurich.html                         1
/Restaurant_Review-g274887-d809556-Reviews-Sandokan_Lisboa_Solingbar-Budapest_Central_Hungary.html      1
/Restaurant_Review-g186338-d4945233-Reviews-Re

In [110]:
# проверяем м столбец с ID_TA (идентификатор ресторана в базе данных TripAdvisor)

base.ID_TA = [i[1:] for i in base.ID_TA]
base['ID_TA'].value_counts()

# так как в предыдущем шаге мы удалили все дубликаты, то в этом у нас все ок

7074681     1
5800094     1
4324960     1
3535470     1
12274149    1
           ..
3411752     1
1163841     1
7337179     1
2014748     1
3616967     1
Name: ID_TA, Length: 39034, dtype: int64

In [111]:
base[col].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39034 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      39034 non-null  int64  
 1   City               39034 non-null  int64  
 2   Cuisine_Style      39034 non-null  int64  
 3   Ranking            39034 non-null  float64
 4   Rating             39034 non-null  float64
 5   Price_Range        39034 non-null  float64
 6   Number_of_Reviews  39034 non-null  object 
 7   Reviews            39034 non-null  object 
 8   URL_TA             39034 non-null  object 
 9   ID_TA              39034 non-null  object 
dtypes: float64(3), int64(3), object(4)
memory usage: 3.3+ MB


In [112]:
base = base.drop('URL_TA', axis=1)
base = base.drop('Reviews', axis=1)
base = base.drop('ID_TA', axis=1)

In [113]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)

from sklearn import metrics  # инструменты для оценки точности модели
# инструмент для создания и обучения модели
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
base = base.fillna(0)


X = base.drop(['Restaurant_id', 'Rating'], axis=1)
y = base['Rating']

# Загружаем специальный инструмент для разбивки:

# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Импортируем необходимые библиотеки:

# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.21194487140075827


In [ ]:
0.21194487140075827